In [1]:
import numpy as np
import random

In [2]:
class Knapsack:
    
    def __init__(self, size, weight_limit, samples):
        self.size = size
        self.weight_limit = weight_limit
        self.samples = samples
        self.genome = np.random.randint(0, 2, (samples, size))
        self.weights = np.random.randint(0, weight_limit/4, size)
        self.values = np.random.randint(0, 101, size)
    
    def __str__(self):
        return self.genome
    
    def fitness(self, idx):
        total_weight = np.sum(self.weights * self.genome[idx, :])
        if total_weight > self.weight_limit:
            return -1
        return np.sum(self.values * self.genome[idx, :])
    
    def batch_fitness(self):
        res = np.zeros(self.samples)
        for idx, row in enumerate(self.genome):
            res[idx] = self.fitness(idx)
        return res
    
    def set_gene(self, row, col, value):
        self.genome[row, col] = value
    
    def set_genome(self, row, value):
        self.genome[row] = value
        
    def reset_genome(self):
        self.genome = np.random.randint(0, 2, (self.samples, self.size))
    
    def mutate(self, idx, current):
        val = random.randint(0, 1)
        if val == current:
            return self.mutate(idx, current)
        return val

In [23]:
class Simulation:
    
    def __init__(self, population, parent_proportion, mutation_rate):
        
        self.population = population
        self.num_parents = int(parent_proportion * self.population.samples)
        self.num_children = self.population.samples - self.num_parents
        self.mutation_rate = mutation_rate
        self.gen = 0
    
    def __str__(self):
        
        return 'Generation: {}\n'.format(self.gen) + str(self.population.genome)
    
    def mating_pool(self):
        
        parents_idx = np.argsort(self.population.batch_fitness())[::-1]
        return parents_idx[:self.num_parents]
    
    def crossover_mid(self, mating_pool):
        
        crossover_point = self.population.size // 2
        children = np.zeros((self.num_children, self.population.size))
        
        for i in range(self.num_children):
            par_idx = np.random.choice(mating_pool, 2, replace=False)
            par0 = self.population.genome[par_idx[0], :crossover_point]
            par1 = self.population.genome[par_idx[1], crossover_point:]
            child_genome = np.concatenate((par0, par1))
            children[i, :] = child_genome
    
        return children
        
    def crossover_random(self, mating_pool):
        
        children = np.zeros((self.num_children, self.population.size))
        cols = np.arange(self.population.size)
        
        for i in range(self.num_children):
            par_idx = np.random.choice(mating_pool, 2, replace=False)
            parents = self.population.genome[par_idx]
            indices = np.random.randint(0, 2, self.population.size)
            children[i, :] = parents[indices, cols]
        
        return children
    
    def random_mutation(self, children):
        
        for i, child in enumerate(children):
            j = random.randint(0, self.population.size - 1)
            children[i, j] = self.population.mutate(j, children[i, j])
    
    def prob_mutation(self, children):
        
        for i, child in enumerate(children):
            for j, val in enumerate(child):
                if random.random() < self.mutation_rate:
                    children[i, j] = self.population.mutate(j, children[i, j])
            
    def generate_genome(self):
        
        mating_pool = self.mating_pool()
        parents = set(mating_pool)
        #children = self.crossover_mid(mating_pool)
        children = self.crossover_random(mating_pool)
        
        # self.random_mutation(children)
        self.prob_mutation(children)
        
        genome = np.zeros((self.population.samples, self.population.size))
        child_idx = 0
        
        for i, row in enumerate(genome):
            if i in parents:
                genome[i, :] = self.population.genome[i, :]
            else:
                genome[i, :] = children[child_idx, :]
                child_idx += 1
        
        return genome

In [11]:
knapsack = Knapsack(30, 200, 150)

In [24]:
simulation = Simulation(knapsack, .1, .1)

In [25]:
simulation.population.reset_genome()

In [26]:
for mutation_rate in [.01, .02, .03, .04, .05, .075, .1, .125, .15, .175, .2, .25, .3, .4, .5]:
    generations = []
    bests = []
    for trial in range(5):
        simulation = Simulation(knapsack, .1, mutation_rate)
        simulation.population.reset_genome()
        best, best_gen = 0, 0
        for i in range(100):
            val = np.max(simulation.population.batch_fitness())
            if val > best:
                best = val
                best_gen = i
            simulation.population.genome = simulation.generate_genome()
        bests.append(best)
        generations.append(best_gen)
    print('Mutation Rate: {}, Generation: {}, Best: {}'.format(mutation_rate, np.mean(generations), np.mean(bests)))

Mutation Rate: 0.01, Generation: 12.8, Best: 804.2
Mutation Rate: 0.02, Generation: 32.2, Best: 810.2
Mutation Rate: 0.03, Generation: 13.0, Best: 819.2
Mutation Rate: 0.04, Generation: 40.2, Best: 815.0
Mutation Rate: 0.05, Generation: 17.4, Best: 821.0
Mutation Rate: 0.075, Generation: 20.0, Best: 820.2
Mutation Rate: 0.1, Generation: 34.4, Best: 820.6
Mutation Rate: 0.125, Generation: 32.2, Best: 821.0
Mutation Rate: 0.15, Generation: 46.4, Best: 820.2
Mutation Rate: 0.175, Generation: 68.6, Best: 819.6
Mutation Rate: 0.2, Generation: 70.0, Best: 816.6
Mutation Rate: 0.25, Generation: 66.0, Best: 792.4
Mutation Rate: 0.3, Generation: 54.0, Best: 759.4
Mutation Rate: 0.4, Generation: 64.4, Best: 723.4
Mutation Rate: 0.5, Generation: 28.4, Best: 661.0


In [66]:
for mutation_rate in [.01, .02, .03, .04, .05, .075, .1, .125, .15, .175, .2, .25, .3, .4, .5]:
    generations = []
    bests = []
    for trial in range(5):
        simulation = Simulation(knapsack, 30, .1, mutation_rate)
        simulation.population.reset_genome()
        best, best_gen = 0, 0
        for i in range(100):
            val = np.max(simulation.population.batch_fitness())
            if val > best:
                best = val
                best_gen = i
            simulation.population.genome = simulation.generate_genome()
        bests.append(best)
        generations.append(best_gen)
    print('Mutation Rate: {}, Generation: {}, Best: {}'.format(mutation_rate, np.mean(generations), np.mean(bests)))

Mutation Rate: 0.01, Generation: 76.0, Best: 1008.0
Mutation Rate: 0.02, Generation: 39.2, Best: 1004.0
Mutation Rate: 0.03, Generation: 34.0, Best: 1012.6
Mutation Rate: 0.04, Generation: 25.6, Best: 1008.2
Mutation Rate: 0.05, Generation: 41.0, Best: 1015.8
Mutation Rate: 0.075, Generation: 26.2, Best: 1015.8
Mutation Rate: 0.1, Generation: 36.4, Best: 1019.0
Mutation Rate: 0.125, Generation: 30.8, Best: 1019.0
Mutation Rate: 0.15, Generation: 66.8, Best: 1019.0
Mutation Rate: 0.175, Generation: 60.8, Best: 1012.2
Mutation Rate: 0.2, Generation: 54.2, Best: 1012.2
Mutation Rate: 0.25, Generation: 67.4, Best: 991.0
Mutation Rate: 0.3, Generation: 58.4, Best: 956.8
Mutation Rate: 0.4, Generation: 35.6, Best: 891.0
Mutation Rate: 0.5, Generation: 56.8, Best: 833.2


In [49]:
def knapSack(W, wt, val, n): 
    K = [[0 for x in range(W + 1)] for x in range(n + 1)] 
  
    # Build table K[][] in bottom up manner 
    for i in range(n + 1): 
        for w in range(W + 1): 
            if i == 0 or w == 0: 
                K[i][w] = 0
            elif wt[i-1] <= w: 
                K[i][w] = max(val[i-1] 
                          + K[i-1][w-wt[i-1]],   
                              K[i-1][w]) 
            else: 
                K[i][w] = K[i-1][w] 
  
    return K[n][W] 
  
val = simulation.population.values
wt = simulation.population.weights
W = 200
n = len(val) 
print(knapSack(W, wt, val, n)) 

697
